In [ ]:
import copy

In [ ]:
graph = {}
with open("graph_dijkstra.txt", "r") as f:
    line = f.readline()
    while line:
        node_edges = line.split()
        node = node_edges[0]
        edges = node_edges[1:]
        # graph[node] = set(edges)
        graph[node] = edges
        line = f.readline()
graph

In [ ]:
# graph = {
#     "A": {"B", "C", "D"},
#     "B": {"A"},
#     "C": {"A"},
#     "D": {"A"}
# }
nodes = list(graph.keys())
node_positions = {v: i for i, v in enumerate(nodes)}
print("Graph:")
graph

In [ ]:
degree_of_nodes = {n: 2 for n in nodes}

print("Degree of all nodes (starting from 0):")
degree_of_nodes # start from 0

In [ ]:
configurations = {
    tuple([0 for i in range(len(nodes))])
}
# perturb each state at a time for all states in configurations and accumulate the same in the configurations for next state to perturb
for n in nodes:
    node_pos = node_positions[n]
    config_copy = copy.deepcopy(configurations)
    for i in range(1, degree_of_nodes[n]+1):
        for cc in config_copy:
            cc = list(cc)
            cc[node_pos] = i
            configurations.add(tuple(cc))
            
print("All possible configurations:")
configurations, len(configurations)

In [ ]:
def check_if_only_one_eligible_process(state):
    """check invariant"""
    bottom = 0
    top = len(state) - 1
    eligible_nodes = 0
    for i, node_state in enumerate(state):
        if i == bottom:
            if (node_state + 1) % 3 == state[i + 1]:
                eligible_nodes += 1

        elif i == top:
            if state[i - 1] == state[0] and (state[i - 1] + 1) % 3 != node_state:
                eligible_nodes += 1

        elif (node_state + 1) % 3 == state[i - 1] or (node_state + 1) % 3 == state[
            i + 1
        ]:
            eligible_nodes += 1

    return eligible_nodes == 1

In [ ]:
invariants = set()
for state in configurations:
    # dijkstra specific
    if check_if_only_one_eligible_process(state):
        invariants.add(state)

print("Invariants and Count of Invariants:")
invariants, len(invariants)

In [ ]:
program_transitions_rank = {}
for inv in invariants:
    program_transitions_rank[inv] = {"L": 0, "C": 1, "A": 0, "Ar": 0, "M": 0}

print("Program transitions rank for invariants:")
program_transitions_rank

In [ ]:
def is_program_transition(perturb_pos, start_state, dest_state):
    # dijkstra specific
    s_prev = start_state[perturb_pos]
    s_new = dest_state[perturb_pos]

    node = nodes[perturb_pos]

    neighbor_pos = [node_positions[n] for n in graph[node]]
    neighbor_states = [start_state[i] for i in neighbor_pos]
    left_state, right_state = neighbor_states

    if node == nodes[0]:  # bottom
        return (s_prev + 1) % 3 == right_state and s_new == (s_prev - 1) % 3

    elif node == nodes[-1]:  # top
        return (
            left_state == right_state
            and (left_state + 1) % 3 != s_prev
            and s_new == (left_state + 1) % 3
        )

    else:  # others
        if (s_prev + 1) % 3 == left_state:
            return s_new == left_state
        elif (s_prev + 1) % 3 == right_state:
            return s_new == right_state
    return False


# is_program_transition(0, [2, 0, 1])

In [ ]:
def get_program_transitions_cvfs(start_state):
    # dijkstra specific
    program_transitions = set()
    cvfs = set()
    for position, _ in enumerate(start_state):
        possible_node_colors = set(range(degree_of_nodes[nodes[position]]+1))
        for perturb_val in possible_node_colors:
            perturb_state = list(start_state)
            perturb_state[position] = perturb_val
            perturb_state = tuple(perturb_state)
            if perturb_state != start_state:
                if is_program_transition(position, start_state, perturb_state):
                    program_transitions.add(perturb_state)
                else:
                    cvfs.add(perturb_state)
    return {"program_transitions": program_transitions, "cvfs": cvfs}

In [ ]:
program_transitions_n_cvf = {}

for state in configurations:
    program_transitions_n_cvf[state] = get_program_transitions_cvfs(state)

print("All Program transitions and CVFs:")
program_transitions_n_cvf

In [ ]:
# program_transitions_n_cvf[(0, 1, 2)]

In [ ]:
unranked_states = set(program_transitions_n_cvf.keys()) - set(program_transitions_rank.keys())
print("Unranked states for Program transitions:")
unranked_states, len(unranked_states)

In [ ]:
# rank the states that has all the paths to the ranked one
while unranked_states:
    ranked_states = set(program_transitions_rank.keys())
    remove_from_unranked_states = set()
    for state in unranked_states:
        dests = program_transitions_n_cvf[state]['program_transitions']
        if dests - ranked_states:       # some desitnations states are yet to be ranked
            pass
        else:                           # all the destination has been ranked
            total_path_length = 0
            path_count = 0
            _max = 0
            for succ in dests:
                path_count += program_transitions_rank[succ]["C"]
                total_path_length += program_transitions_rank[succ]["L"] + program_transitions_rank[succ]["C"]
                _max = max(_max, program_transitions_rank[succ]["M"])
            program_transitions_rank[state] = {
                "L": total_path_length,
                "C": path_count,
                "A": total_path_length/path_count,
                "Ar": round(total_path_length/path_count),
                "M": _max + 1
            }
            remove_from_unranked_states.add(state)
    print(remove_from_unranked_states)
    unranked_states -= remove_from_unranked_states

In [ ]:
unranked_states

In [ ]:
print("Program transitions Rank :")
program_transitions_rank

In [ ]:
pt_rank_effect = {}
for state, transition_cvfs in program_transitions_n_cvf.items():
    for pt in transition_cvfs['program_transitions']:
        pt_rank_effect[(state, pt)] = {
            "A": program_transitions_rank[pt]["A"] - program_transitions_rank[state]["A"],
            "M": program_transitions_rank[pt]["M"] - program_transitions_rank[state]["M"]
        }
        pt_rank_effect[(state, pt)]["Ar"] = round(pt_rank_effect[(state, pt)]["A"])

In [ ]:
cvfs_rank = {}
for state, transition_cvfs in program_transitions_n_cvf.items():
    for cvf in transition_cvfs['cvfs']:
        cvfs_rank[(state, cvf)] = {
            "A": program_transitions_rank[cvf]["A"] - program_transitions_rank[state]["A"],
            "M": program_transitions_rank[cvf]["M"] - program_transitions_rank[state]["M"]
        }
        cvfs_rank[(state, cvf)]["Ar"] = round(cvfs_rank[(state, cvf)]["A"])

In [ ]:
print("CVFs Rank :")
cvfs_rank

In [ ]:
pt_avg = [i["Ar"] for _, i in pt_rank_effect.items()]
pt_max = [i["M"] for _, i in pt_rank_effect.items()]

In [ ]:
cvfs_avg = [i["Ar"] for _, i in cvfs_rank.items()]
cvfs_max = [i["M"] for _, i in cvfs_rank.items()]

In [ ]:
from matplotlib import pyplot as plt

n, bins, patches = plt.hist(pt_avg, bins=len(set(pt_avg)))
ticks = [(patch._x0 + patch._width/2) for patch in patches]
ticklabels = sorted([i for i in set(pt_avg)])
plt.xticks(ticks, ticklabels)
plt.show()

In [ ]:
n, bins, patches = plt.hist(pt_max, bins=len(set(pt_max)))
ticks = [(patch._x0 + patch._width/2) for patch in patches]
ticklabels = sorted([i for i in set(pt_max)])
plt.xticks(ticks, ticklabels)
plt.show()

In [ ]:
n, bins, patches = plt.hist(cvfs_avg, bins=len(set(cvfs_avg)))
ticks = [(patch._x0 + patch._width/2) for patch in patches]
ticklabels = sorted([i for i in set(cvfs_avg)])
plt.xticks(ticks, ticklabels)
plt.show()

In [ ]:
n, bins, patches = plt.hist(cvfs_max, bins=len(set(cvfs_max)))
ticks = [(patch._x0 + patch._width/2) for patch in patches]
ticklabels = sorted([i for i in set(cvfs_max)])
plt.xticks(ticks, ticklabels)
plt.show()